<a href="https://colab.research.google.com/github/Harikaraja/News_headline_Generation-using-NLP/blob/Final-Integration/Final_Integration_%3E_26_March_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
!wget https://nlp.stanford.edu/software/stanford-corenlp-4.5.6.zip

!unzip stanford-corenlp-4.5.6.zip

--2024-03-26 12:10:28--  https://nlp.stanford.edu/software/stanford-corenlp-4.5.6.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://downloads.cs.stanford.edu/nlp/software/stanford-corenlp-4.5.6.zip [following]
--2024-03-26 12:10:28--  https://downloads.cs.stanford.edu/nlp/software/stanford-corenlp-4.5.6.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 505656981 (482M) [application/zip]
Saving to: ‘stanford-corenlp-4.5.6.zip’

stanford-corenlp-4. 100%[===================>] 482.23M  5.00MB/s    in 91s     

2024-03-26 12:11:59 (5.29 MB/s) - ‘stanford-corenlp-4.5.6.zip’ saved [505656981/505656981]

Archive:  stanford-corenlp-4.5.6.zip
  

In [3]:
!pip install textacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 29.6 MB/s eta 0:00:00


In [7]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
import string
import os
from nltk.parse.corenlp import CoreNLPServer
from nltk.parse.corenlp import CoreNLPParser
from nltk.corpus import stopwords
from nltk.tree.tree import Tree
import re
import textacy
from textacy import *
from gensim.models import Word2Vec
from scipy.spatial import distance
import networkx as nx


In [9]:
os.environ['CLASSPATH'] = 'stanford-corenlp-4.5.6'  #setting environment variable

In [24]:
class News_headline_generation:

    def __init__(self, df):
        self.df = df
        self.sentences = sent_tokenize(self.df)

    def pre_process(self):
        sentences_clean = [re.sub(r'[^\w\s]', '', sentence.lower()) for sentence in self.sentences]
        stop_words = stopwords.words('english')
        sentence_tokens = [[words for words in sentence.split(' ') if words not in stop_words] for sentence in sentences_clean]
        return sentence_tokens

    # def count_paragraphs(self):
    #     val=self.df
    #     paragraphs = re.split(r"\n\n+",val)
    #     num_paragraphs = len(paragraphs)
    #     print("num_paragraphs: ",num_paragraphs)

    def count_paragraphs(self):
        text=self.df
        paragraphs = re.split(r'\n\s*\n', text)
        return (paragraphs,len(paragraphs))

    def word2vec(self):
        sentence_tokens = self.pre_process()
        w2v = Word2Vec(sentence_tokens, vector_size=1, min_count=1, epochs=1000)
        sentence_embeddings = []
        max_len = max(len(tokens) for tokens in sentence_tokens)
        for words in sentence_tokens:
            embedding = [w2v.wv[word] for word in words]
            padding_length = max_len - len(embedding)
            padded_embedding = np.pad(embedding, [(0, padding_length), (0, 0)], mode='constant')
            sentence_embeddings.append(padded_embedding)
        return sentence_embeddings

    def similarity_matrix(self):
        sentence_tokens = self.pre_process()
        sentence_embeddings = self.word2vec()
        similarity_matrix = np.zeros([len(sentence_tokens), len(sentence_tokens)])
        for i, row_embedding in enumerate(sentence_embeddings):
            for j, column_embedding in enumerate(sentence_embeddings):
                similarity_matrix[i][j] = 1 - distance.cosine(row_embedding.ravel(), column_embedding.ravel())
        return similarity_matrix

    def num_of_leadingsentences(self):
        num_sentences = len(self.sentences)
        if num_sentences < 5:
            top = 1
        elif num_sentences < 10:
            top = 2
        elif num_sentences < 25:
            top = 4
        elif num_sentences < 50:
            top = 9
        elif num_sentences < 100:
            top = 18
        elif num_sentences < 200:
            top = 25
        elif num_sentences >= 201:
            top = 40
        return top

    def text_rank(self,num_sentences_to_extract):
        li=[]
        similarity_matrixs = self.similarity_matrix()
        nx_graph = nx.from_numpy_array(similarity_matrixs)
        scores = nx.pagerank(nx_graph)
        top_sentence = {sentence: scores[index] for index, sentence in enumerate(self.sentences)}
        top = dict(sorted(top_sentence.items(), key=lambda x: x[1], reverse=True)[:num_sentences_to_extract])
        for sent in self.sentences:
            if sent in top.keys():
                li.append(sent)
        return li

    def leading_sentences(self):
        article_info = self.count_paragraphs()
        leading_sentences=[]
        #if there is only one para in article then num_of_leading sentences are selected based on fixed constant
        if article_info[1]==1:
          num_sentences_to_extract=self.num_of_leadingsentences()
          LSG_article = News_headline_generation(str(article_info[0]))
          leading_sentences.extend(LSG_article.text_rank(num_sentences_to_extract))
          #leading_sentences_corpus.append(leading_sentences)
        else:
          num_sentences_to_extract=1                   #if there are more than one paras in article
          paragraphs = article_info[0]
          #print("num_paras: ",paragraphs)
          #extracting one leading sentence from each paragraph
          for para in paragraphs:
              LSG = News_headline_generation(para)
              output = LSG.text_rank(num_sentences_to_extract)
              leading_sentences.extend(output)
          #extractig leading sentence from entire article
          LSG_article = News_headline_generation(para)
          leading_sentences.extend(LSG_article.text_rank(num_sentences_to_extract))

        return leading_sentences

    def Parsing(self,Sentence, server):
        parser = CoreNLPParser(url=server.url)
        return next(parser.raw_parse(Sentence))

    def find_leftmost_S(self,tree):
        if isinstance(tree, str):  # Terminal node
            return None
        elif tree.label() == 'S':  # Found leftmost S node
            return tree
        else:
            for subtree in tree:
                result = self.find_leftmost_S(subtree)
                if result is not None:
                    return result

    def Pruning(self,tree, Label):
          if isinstance(tree, str):
            return tree
          if tree.height() > 0:
            filtered_children = [self.Pruning(child, Label) for child in tree if (isinstance(child, str) or child.height() > 0) and (isinstance(child, str) or child.label() != Label)]
            return Tree(tree.label(), filtered_children)
          else:
            return tree

    def Extract(self,tree):
          words = []
          for subtree in tree:
              if isinstance(subtree, Tree):
                  words.extend(self.Extract(subtree))
              else:
                  words.append(subtree)
          return words

    def low_content_words(word_list):
          stop_words = set(stopwords.words('english'))
          words = [word.lower() for word in word_list]
          filtered_li=[]
          for word in words:
            if word not in stop_words:
              filtered_li.append(word)
          #print("filtered_li: ",filtered_li)
          word_freq = Counter(filtered_li)
          total_words = len(words)
          low_content_threshold = 0.01
          low_content_words = {word for word, freq in word_freq.items() if freq >= low_content_threshold * total_words}
          return low_content_words

    def Matching(TFdict, text):
          translator = str.maketrans('', '', string.punctuation)
          text = text.translate(translator)
          words = word_tokenize(text)

          porter_stemmer = PorterStemmer()
          stemmed_words = [porter_stemmer.stem(word.lower()) for word in words]

          count = 0
          for i in stemmed_words:
            if i in TFdict.keys():
              count += TFdict[i]
          return count

    def SGRMatching(self, HeadLine, TopPhrases):
          l, Flag, itre = len(TopPhrases), 0.0, 0
          for Phrase in TopPhrases:
            if Phrase in HeadLine:
              Flag += (l - TopPhrases.index(Phrase)) / l
              itre += 1
              #print(f"{Phrase} : {TopPhrases.index(Phrase)}")
          #print(f"Cumiliative Value : {Flag / itre}")
          if itre != 0:
            return Flag / itre
          else:
            return -1

    def KeyPhraseSGRank(self):
          en = textacy.load_spacy_lang("en_core_web_sm")

          doc = textacy.make_spacy_doc(self.df, lang=en)

          TopPhrases = [kps for kps, weights in textacy.extract.keyterms.sgrank(doc, topn=1.0)]

          return TopPhrases

    def Generate(self):
        CompressedSentences = []
        server = CoreNLPServer()
        server.start()
        leading_sentences_list = self.leading_sentences()
        for i in leading_sentences_list:
          ParsedSentence = self.Parsing(i, server)
          for i in ParsedSentence:
            for j in i:
              lefts = self.find_leftmost_S(j)
              if lefts is not None:
                LeftMostS = lefts
              else:
                LeftMostS = i
              break
          Labels = ['DT', 'TMP', 'SBAR']
          #, 'CC', 'VBZ, 'PRP'' IN',
          for i in Labels:
            Temp = self.Pruning(LeftMostS, i)
            LeftMostS = Temp
          word_list = self.Extract(Temp)
          #low_content_words_inpara=low_content_words(word_list)
          sentence = ' '.join(word_list)
          CompressedSentences.append(sentence)
        server.stop()

        ResultDict = {}
        KeyPhrases = self.KeyPhraseSGRank()
        for i in CompressedSentences:
          #ResultDict[i] = Matching(TFdict, i)
          ResultDict[i] = self.SGRMatching(i, KeyPhrases)

        #print(ResultDict)

        max_key = max(ResultDict, key=lambda k: ResultDict[k])
        #print("max_key: ",max_key)
        return max_key

**EXAMPLE 1**

In [22]:
Article = """IDH’s Life and Building Safety (LABS) Initiative is delighted to announce a partnership with Nike, a leading athletic footwear, apparel, and accessories manufacturer. This collaboration is a significant step forward in the ongoing commitment to fostering safer working conditions and sustainable practices within the global supply chain.

As part of this collaboration, the LABS Initiative will work closely with factories supplying to Nike to enhance safety standards, ensuring they undergo comprehensive assessments and safety training processes.

LABS brings together industry leaders, non-governmental organizations, and stakeholders to collectively address and enhance life and building safety in factories worldwide. Nike’s decision to join the LABS Initiative signifies a shared commitment to elevating safety standards in manufacturing facilities and promoting responsible and sustainable business practices and further strengthens the foundation pillar for the program.

The LABS Initiative is excited to embark on this journey with Nike, collectively driving positive change and setting new benchmarks for safety and sustainability in the athletic apparel and footwear industry."""

In [25]:
NHG = News_headline_generation(Article)
output = NHG.Generate()
print("Output is: ",output)

Output is:  As part of collaboration , LABS Initiative will work closely with factories supplying to Nike to enhance safety standards , ensuring .


**EXAMPLE 2**

In [26]:
Article = """Einar Bessi Gestsson, a natural disaster expert at the Norwegian Meteorological Agency, has told Iceland's public broadcaster RUV that dangerous gases and small explosions could occur if lava makes contact with sea water.

Meanwhile, the lava moving west is heading in the direction of the Blue Lagoon and a geothermal power plant, which provides hot water for most of the Reykjanes Peninsula.

The Icelandic Met Office said this lava bed was "significantly wider" than in February, when an earlier eruption caused lava to flow in a similar direction.

Many protective embankments have been built around both, the head of the Reykjavik-based Nordic Volcanological Centre, Rikke Pedersen, told Reuters.

There are concerns that fibre optic cables on the road could be damaged - causing disruption to phone and internet services."""

In [27]:
NHG = News_headline_generation(Article)
output = NHG.Generate()
print("Output is: ",output)

Output is:  Icelandic Met Office said .


**EXAMPLE 3**

In [30]:
Article = """Qatar's foreign ministry spokesman says the passing on Monday of a UN Security Council resolution calling for a ceasefire in Gaza has had no "immediate impact" on the ongoing negotiations between Israel and Hamas.

These talks, which are aimed at agreeing a temporary truce and the release of hostages still being held by Hamas, are currently taking place in Doha with the input of mediators from Qatar, Egypt and the US.

According to the Reuters news agency, sources briefed on the discussions said they were "moving ahead" and that while some of the Israeli delegation has left Doha, a smaller group of representatives remains.

Israel's Haaretz newspaper also cited an Israeli source as stating that "the deal has not collapsed".

However, Israeli Prime Minister Benjamin Netanyahu warned in a statement that the UN resolution had "damaged" the negotiations and encouraged Hamas to stick to its "delusional demands".

Hamas says a ceasefire should mean the end of the war and the withdrawal of Israeli forces from Gaza, which Netanyahu rejects."""

#original headline: UN ceasefire resolution has had no 'immediate impact' on talks - Qatar

In [31]:
NHG = News_headline_generation(Article)
output = NHG.Generate()
print("Output is: ",output)



Output is:  Qatar 's foreign ministry spokesman says .


**Example 4**

In [32]:
Article = """As we’ve been reporting, fighting is continuing in Gaza - despite the UN Security Council resolution calling for a ceasefire being passed for the first time on Monday.

Ohad Tal is an Israeli MP from the far-right Religious Zionism party, which is part of Prime Minister Benjamin Netanyahu's coalition.

Speaking to the BBC World Service’s Newsday programme earlier, he was asked why Israel is continuing its offensive in Gaza in the face of such a resolution.

“We have a right to defend ourselves – it’s as simple as that,” Tal replied.

He said the world must "make a choice" between supporting Israel or Hamas."""

#original headline: We have a right to defend ourselves – Israeli MP

In [33]:
NHG = News_headline_generation(Article)
output = NHG.Generate()
print("Output is: ",output)

Output is:  Speaking to BBC World Service ’s Newsday programme earlier
